<a href="https://colab.research.google.com/github/urosdedic/predikcija-kategorije-proizvoda/blob/main/predikcija_kategorije_proizvoda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predikcija kategorije proizvoda prema naslovu